In [1]:
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc
import dash_bio as dashbio
import pandas as pd
import plotly.express as px
import numpy as np
import dash_bootstrap_components as dbc

In [2]:
app = Dash(external_stylesheets=[dbc.themes.PULSE])

In [3]:
df = pd.read_csv('https://git.io/manhattan_data.csv')
df['-log10p'] = -np.log10(df['P'])
df.head()

,CHR,BP,P,SNP,ZSCORE,EFFECTSIZE,GENE,DISTANCE,-log10p
0,1,937641,0.335344,rs9697358,0.9634,-0.0946,ISG15,1068,0.474510
1,1,1136887,0.245857,rs34945898,1.1605,-0.0947,TNFRSF4,0,0.609317
2,1,2116240,0.823286,rs12034613,0.2233,-0.0741,FP7162,0,0.084449
3,1,2310562,0.493204,rs4648633,0.6852,0.0146,MORN1,0,0.306974
4,1,2681715,0.605392,rs4430271,0.5167,0.1234,MMEL1,127427,0.217964


In [4]:
sub_df = df[df['CHR'] == 1]

In [5]:
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "20rem",
    "padding": "2rem 1rem",
    'background-color': '592c98'
}

CONTENT_STYLE = {
    "margin-left": "25rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}


In [6]:
sidebar = html.Div(
    [
        html.H2('Joseph Staniforth Dash Portfolio', className="display-4"),
        html.Hr(),
        html.P(
            'Controls for the two plots on the right', className="lead"
        ),
        dbc.Label('Chromosome Scatter Plot'),
        dcc.Dropdown([*range(1,24,1)], 1, id='chr_drop_down',
                     style = {'display': 'inline-block', 'width': '12em'}),
        html.Br(),
        dbc.Label('Manhattan Plot'),
        dcc.RadioItems(['-log10 p', 'Raw p-value'], '-log10 p', id='log_radio',
                       labelStyle={'display': 'block'}),
        html.Br(),
        dcc.Slider(1, 9, 1, value = 5, id = 'p_slider')
    ],
    style=SIDEBAR_STYLE,
)

In [7]:
content = html.Div([
    html.Br(),
    html.Div([
        dcc.Graph(id='partial_scatter',
                  figure = px.scatter(sub_df, x = 'BP', y = '-log10p', color_discrete_sequence=['#20B2AA'], title = 'Chromosome 1'),
                  style={'display': 'inline-block', 'width':'55vh'}),
        dcc.Graph(
            id='full_manhattan',
            figure=dashbio.ManhattanPlot(
                dataframe=df,
                suggestiveline_value = False,
                highlight_color = '#20B2AA',
                showlegend = False,
            showgrid = False), 
            style={'display': 'inline-block', 'width':'83vh'})
    ],style={'display': 'inline-block'}),
], style=CONTENT_STYLE)


In [8]:
app.layout = html.Div([sidebar, content])

In [9]:
@app.callback(
    Output('full_manhattan', 'figure'),
    Input('p_slider', 'value'),
    Input('log_radio', 'value')
)

def update_manhattanplot(p_slider, log_radio):
    return dashbio.ManhattanPlot(
        dataframe=df,
        genomewideline_value=p_slider,
        suggestiveline_value = False,
        highlight_color = '#20B2AA',
        showlegend = False,
        logp = True if log_radio == '-log10 p' else False,
        showgrid = False
    )

@app.callback(
    Output("partial_scatter", "figure"), 
    Input("chr_drop_down", "value"))

def update_bar_chart(chr_drop_down):
    mask = df[df['CHR'] == chr_drop_down]
    fig = px.scatter(mask, x = 'BP', y = '-log10p', color_discrete_sequence=['#20B2AA'], title = f'Chromosome {chr_drop_down}')
    return fig


In [10]:
app.run_server(port = 8051)

Dash is running on http://127.0.0.1:8051/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8051 (Press CTRL+C to quit)


![title](ezgif-1-46c3c96bd2.gif)